In [559]:
import pandas as pd
import numpy as np
import pymc as pm
from sklearn.cluster import KMeans
import random
from sklearn.preprocessing import StandardScaler

In [394]:
flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")
flipped

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/3063244417.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")


,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end,Right_end,Flip
0,202109090097,41.56,23.92,2021090900,97,53441.0,6,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.12,-1.99,none,40.0,53441.0,42403.0,1
1,202109090097,41.56,23.92,2021090900,97,53441.0,7,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.06,-1.96,none,40.0,53441.0,42403.0,1
2,202109090097,41.56,23.92,2021090900,97,53441.0,8,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.96,-1.91,none,40.0,53441.0,42403.0,1
3,202109090097,41.56,23.92,2021090900,97,53441.0,9,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.80,-1.85,none,40.0,53441.0,42403.0,1
4,202109090097,41.56,23.92,2021090900,97,53441.0,10,2021-09-10 00:26:32,11.0,DAL,...,TRADITIONAL,0,6.0,1.56,-1.72,none,40.0,53441.0,42403.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374366,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,7.18,3.67,none,44.0,37097.0,46083.0,0
1374367,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,7.22,3.89,none,44.0,37097.0,46083.0,0
1374368,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,7.0,7.27,4.08,none,44.0,37097.0,46083.0,0
1374369,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,7.0,7.31,4.26,none,44.0,37097.0,46083.0,0


In [395]:
frames_per_play = flipped[flipped['pff_positionLinedUp']=='QB'].groupby('uniqueplayId', as_index=False).size().rename(columns={'size':'play_length_frames'})

In [396]:
flipped = flipped.merge(frames_per_play, on='uniqueplayId', how='left')

In [397]:
flipped['play_length_frames'].min()

16.0

In [398]:
flipped.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'gameId', 'playId', 'nflId',
       'frameId', 'time', 'jerseyNumber', 'team', 'playDirection', 'x', 'y',
       's', 'a', 'dis', 'o', 'dir', 'event', 'pff_role', 'pff_positionLinedUp',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'dropBackType',
       'pff_playAction', 'ball_snap_frame', 'new_x', 'new_y', 'chip_side',
       'end_frame', 'Left_end', 'Right_end', 'Flip', 'play_length_frames'],
      dtype='object')

In [399]:
def name_func(pos):
    temp = {'nflId':pos+'_nflId',
     'jerseyNumber':pos+'_jerseyNumber',
     'team':pos+'_team', 
     'x':pos+'_x', 
     'y':pos+'_y',
     's':pos+'_s', 
     'a':pos+'_a', 
     'dis':pos+'_dis', 
     'o':pos+'_o', 
     'dir':pos+'_dir', 
     'pff_role':pos+'_pff_role', 
     'pff_positionLinedUp':pos+'_pff_positionLinedUp',
     'pff_nflIdBlockedPlayer':pos+'_pff_nflIdBlockedPlayer', 
     'pff_blockType':pos+'_pff_blockType', 
     'new_x':pos+'_new_x', 
     'new_y':pos+'_new_y', 
     'Flip':pos+'_Flip'}
    return temp

In [400]:
qb = flipped[(flipped['pff_positionLinedUp']=='QB')]
qb = qb.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('QB')).reset_index(drop=True)

In [401]:
qb_flip = flipped[(flipped['pff_positionLinedUp']=='QB_flip')]
qb_flip = qb_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('QB')).reset_index(drop=True)

In [402]:
ball = flipped[(flipped['pff_positionLinedUp']=='ball')]
ball = ball.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('ball')).reset_index(drop=True)

In [403]:
ball_flip = flipped[(flipped['pff_positionLinedUp']=='ball_flip')]
ball_flip = ball_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('ball')).reset_index(drop=True)


In [404]:
rt = flipped[(flipped['pff_positionLinedUp']=='RT')]
rt = rt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('Tackle')).reset_index(drop=True)

In [405]:
lt = flipped[(flipped['pff_positionLinedUp']=='LT')]
lt = lt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('Tackle')).reset_index(drop=True)

In [406]:
le = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==1)]
le = le.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('End')).reset_index(drop=True)

In [407]:
re = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==0)]
re = re.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('End')).reset_index(drop=True)

In [408]:
play = flipped[(flipped['pff_positionLinedUp']=='QB')][['uniqueplayId', 'football_x', 'football_y', 
                                                        'gameId', 'playId', 'frameId', 'time', 'playDirection', 
                                                        'event', 'ball_snap_frame', 'end_frame', 'play_length_frames',
                                                        'Right_end', 'Left_end']].reset_index(drop=True)

In [409]:
final_left = play.merge(qb, on=['uniqueplayId', 'frameId'], how='left').merge(ball, on=['uniqueplayId', 'frameId'], how='left')

In [410]:
final_left = final_left.merge(lt, on=['uniqueplayId', 'frameId'], how='left').merge(re, on=['uniqueplayId', 'frameId'], how='left')

In [411]:
final_right = play.merge(qb_flip, on=['uniqueplayId', 'frameId'], how='left').merge(ball_flip, on=['uniqueplayId', 'frameId'], how='left')

In [412]:
final_right = final_right.merge(rt, on=['uniqueplayId', 'frameId'], how='left').merge(le, on=['uniqueplayId', 'frameId'], how='left')

In [566]:
final = pd.concat([final_left, final_right])

In [414]:
play_ex = final[final['uniqueplayId']==20210919023392]

Before classifying:<br>
> - Remove frames before ball is snapped and after it is thrown<br>
> - Remove tackle and end where the end's pff_role is Coverage<br>
> - First check: Remove plays where end moves into the inside (stunt)<br>
> - Remove plays where the distance between end and tackle is large directly after ball snap (not against each other)<br>
<br>

### Classifying
##### Clustering into two groups<br>
Predictor variables:<br>
> - Percentage that tackle/end are looking at each other <br>
> - Percentage that Euclidian distance is less than 0.5 (0.75)<br>
> - Total distance traveled of defensive end<br>
> - Max speed during rush<br>

In [567]:
def dist(x_1, y_1, x_2, y_2):
    return np.sqrt(np.sum([(x_1-x_2)**2, (y_1-y_2)**2], axis=0))

In [568]:
final['tackle_end_dist'] = dist(final['Tackle_new_x'], final['Tackle_new_y'], final['End_new_x'], final['End_new_y'])

In [570]:
#angle_range = 20
def check_angle(x, y):
    diff = abs(abs(x-y)-180)
    #if diff <= angle_range:
        #return 1
    #else:
        #return 0
    return diff

In [571]:
final['tackle_end_facing'] = final.apply(lambda row: check_angle(row['Tackle_o'], row['End_o']),  axis=1)

In [572]:
final

,uniqueplayId,football_x,football_y,gameId,playId,frameId,time,playDirection,event,ball_snap_frame,...,End_dir,End_pff_role,End_pff_positionLinedUp,End_pff_nflIdBlockedPlayer,End_pff_blockType,End_new_x,End_new_y,End_Flip,tackle_end_dist,tackle_end_facing
0,202109090097,41.56,23.92,2021090900,97,6,2021-09-10 00:26:31,right,ball_snap,6.0,...,247.75,Pass Rush,ROLB,NaN,NaN,2.34,-8.71,0.0,6.104891,46.42
1,202109090097,41.56,23.92,2021090900,97,7,2021-09-10 00:26:31,right,None,6.0,...,252.79,Pass Rush,ROLB,NaN,NaN,2.27,-8.70,0.0,6.009076,46.23
2,202109090097,41.56,23.92,2021090900,97,8,2021-09-10 00:26:31,right,None,6.0,...,248.14,Pass Rush,ROLB,NaN,NaN,2.12,-8.62,0.0,5.785404,45.79
3,202109090097,41.56,23.92,2021090900,97,9,2021-09-10 00:26:31,right,None,6.0,...,246.73,Pass Rush,ROLB,NaN,NaN,1.93,-8.54,0.0,5.508049,51.35
4,202109090097,41.56,23.92,2021090900,97,10,2021-09-10 00:26:32,right,None,6.0,...,243.55,Pass Rush,ROLB,NaN,NaN,1.55,-8.32,0.0,5.089764,57.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168448,20211025003926,85.95,23.68,2021102500,3926,45,2021-10-26 03:14:08,left,None,7.0,...,26.59,Pass Rush,LOLB,NaN,NaN,4.83,-3.77,1.0,1.314002,35.51
168449,20211025003926,85.95,23.68,2021102500,3926,46,2021-10-26 03:14:08,left,None,7.0,...,23.83,Pass Rush,LOLB,NaN,NaN,4.91,-3.94,1.0,1.338096,46.50
168450,20211025003926,85.95,23.68,2021102500,3926,47,2021-10-26 03:14:09,left,None,7.0,...,21.57,Pass Rush,LOLB,NaN,NaN,4.98,-4.11,1.0,1.362424,48.72
168451,20211025003926,85.95,23.68,2021102500,3926,48,2021-10-26 03:14:09,left,None,7.0,...,18.94,Pass Rush,LOLB,NaN,NaN,5.05,-4.28,1.0,1.373062,48.08


In [573]:
final['end_too_far'] = (final['frameId']>final['ball_snap_frame']+10) &(final['tackle_end_dist']>4)

In [718]:
def full_agg(grouped_df):
    return pd.DataFrame({
    'end_too_far': [1 if grouped_df['end_too_far'].sum()>=1 else 0],
    'Mean_tackle_end_dist': [grouped_df[grouped_df['frameId']>grouped_df['ball_snap_frame']+10]['tackle_end_dist'].mean()],
    'Mean_tackle_end_facing': [grouped_df['tackle_end_facing'].mean()],
    'End_dist_travel': [grouped_df['End_dis'].sum()],
    'Mean_end_speed': [grouped_df['End_s'].mean()]
})

cluster_data = final.groupby(['uniqueplayId', 'QB_Flip'], group_keys=True).apply(full_agg).reset_index().drop(['level_2'], axis=1)

In [719]:
cluster_data

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed
0,202109090097,0.0,0,1.124898,41.940789,13.31,3.191316
1,202109090097,1.0,0,0.928589,73.850263,11.74,3.073684
2,202109120163,0.0,0,0.881869,31.188148,8.49,3.167407
3,202109120163,1.0,0,0.758866,41.957407,7.51,2.728889
4,202109120288,0.0,0,1.174814,114.386087,6.77,2.684783
...,...,...,...,...,...,...,...
9653,20211025003684,1.0,0,1.156339,65.342400,14.49,2.835600
9654,20211025003904,0.0,0,0.660331,22.898710,9.05,2.853548
9655,20211025003904,1.0,0,0.882441,37.075484,9.33,2.933226
9656,20211025003926,0.0,0,0.823808,37.409535,6.47,1.487442


In [689]:
# cluster_data = final.groupby(['uniqueplayId', 'QB_Flip'], as_index=False).agg({
#     'end_too_far': lambda x: 1 if x.sum()>=1 else 0,
#     'tackle_end_dist': 'mean',
#     'tackle_end_facing': 'mean',
#     'End_dis': 'sum',
#     'End_s':'mean'
# }).rename(columns={
#      'tackle_end_dist':'Mean_tackle_end_dist',
#      'tackle_end_facing':'Mean_tackle_end_facing',
#      'End_dis':'End_dist_travel',
#      'End_s':'Mean_end_speed'
#  })
# cluster_data

In [669]:
power = final[(final['uniqueplayId']==2021091902389)&(final['QB_Flip']==0)]
speed = final[(final['uniqueplayId']==20210919023392)&(final['QB_Flip']==0)]
power_clust = cluster_data[(cluster_data['uniqueplayId']==2021091902389)&(cluster_data['QB_Flip']==0)]
speed_clust = cluster_data[(cluster_data['uniqueplayId']==20210919023392)&(cluster_data['QB_Flip']==0)]

In [670]:
power_clust

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed
519,2021091902389,0.0,0,0.813461,13.812619,9.65,2.212143


In [671]:
speed_clust

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed
3551,20210919023392,0.0,0,0.576881,79.314516,12.76,4.015161


In [720]:
cluster_final = cluster_data[cluster_data['end_too_far']==0]

In [727]:
test_plays = [20210926112818,2021092602494,20211010112817,20211017071173,2021101010874,20211017024110, 20211003081723, 2021091901500, 20211021002175, 20210926051887]

cluster_final_test = cluster_final[cluster_final['uniqueplayId'].isin(test_plays)]
cluster_final_test['actual'] = [1,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,0,1,1,1]
#[(i-1)*-1 for i in [1,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,0,1,1,1]]
cluster_final_train = cluster_final[~cluster_final['uniqueplayId'].isin(test_plays)]

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/2559911687.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_final_test['actual'] = [1,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,0,1,1,1]


In [728]:
cluster_X = cluster_final_train.drop(
    ['uniqueplayId', 'QB_Flip', 'end_too_far'], axis=1)

In [729]:
kmeans = KMeans(n_clusters=2, random_state=2023).fit(cluster_X)

In [730]:
cluster_final_test['Cluster'] = kmeans.predict(cluster_final_test.drop(
    ['uniqueplayId', 'QB_Flip', 'end_too_far', 'actual'], axis=1))

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/237906636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_final_test['Cluster'] = kmeans.predict(cluster_final_test.drop(


In [731]:
cluster_final_test

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,actual,Cluster
502,2021091901500,0.0,0,1.126498,19.039167,4.65,1.885833,1,1
503,2021091901500,1.0,0,1.138747,43.272917,9.10,3.792083,0,1
800,2021092602494,0.0,0,0.963851,14.988919,9.52,2.474595,1,1
801,2021092602494,1.0,0,1.290382,80.818378,11.84,3.111622,0,0
1575,2021101010874,0.0,0,0.878565,30.746389,9.99,2.678056,1,1
1576,2021101010874,1.0,0,0.969142,81.063056,13.04,3.513333,0,0
4794,20210926051887,0.0,0,2.254614,45.412727,6.60,2.911364,1,1
4795,20210926051887,1.0,0,0.713270,22.306818,5.76,2.595000,1,1
5303,20210926112818,0.0,0,0.925383,79.243514,11.82,3.231892,0,0
5304,20210926112818,1.0,0,1.345928,49.224865,10.36,2.793784,1,1


In [732]:
# Accuracy
sum(cluster_final_test['actual']==cluster_final_test['Cluster'])/len(cluster_final_test)

0.75

In [638]:
labs = kmeans.labels_
cluster_final_train['rushType'] = labs

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/2913040956.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_final_train['rushType'] = labs


In [584]:
random.seed(11902)
rand_ind = random.sample(range(0, len(cluster_final)), 20)

In [585]:
cluster_final.iloc[rand_ind, :]

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,rushType
8255,20211017063573,1.0,0,1.554623,67.598611,16.01,4.471389,0
8295,20211017071627,0.0,0,1.133557,51.209048,10.71,2.500476,0
3327,20210916001343,1.0,0,1.281122,72.568800,5.31,2.118000,0
8332,20211017073078,1.0,0,1.492179,13.678621,9.15,3.048621,1
7919,20211017012947,0.0,0,1.075878,32.304722,6.46,1.818889,1
4911,20210926063298,1.0,0,1.813869,50.632051,5.59,1.460256,0
2683,20210912053805,1.0,0,1.482480,67.121818,13.40,4.129091,0
1639,2021101400675,0.0,0,1.541845,75.541000,11.49,3.706333,0
5293,20210926112428,0.0,0,0.820183,20.257353,7.19,2.054706,1
6692,20211010001051,0.0,0,0.979687,21.130488,6.80,1.578293,1


In [586]:
cluster_final[cluster_final['uniqueplayId']==20211017063573]

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,rushType
8254,20211017063573,0.0,0,2.208171,16.827778,12.01,3.364444,1
8255,20211017063573,1.0,0,1.554623,67.598611,16.01,4.471389,0


In [709]:
kmeans.cluster_centers_

array([[ 1.07731719, 27.99706921,  8.58015592,  2.69812023,  4.22477224],
       [ 1.13747133, 70.45075449, 10.62403947,  3.02706926,  4.59840113]])

In [531]:
kmeans.get_params()

{'algorithm': 'lloyd',
 'copy_x': True,
 'init': 'k-means++',
 'max_iter': 300,
 'n_clusters': 2,
 'n_init': 10,
 'random_state': 2023,
 'tol': 0.0001,
 'verbose': 0}